## Introduction to Linear Regression

In this we'll discuss one of the foundational algorithms in Machine Learning: *Linear regression*. We'll create a model that predicts crop yields for apples and oranges (*target variable*) by looking at the average temperature, rainfall, and humidity (*input variables or features*) in a region. Here's the training data:
![](https://i.imgur.com/6Ujttb4.png)

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias:

    yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1

    yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2

Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:
![](https://i.imgur.com/4DJ9f8X.png)

The *learning part* of linear regression is to figure out a set of weights `w11, w12,... w23, b1 & b2` using the training data, to make accurate predictions for new data. The learned weights will be used to predict the yields for apples and oranges in a new region using the average temperature, rainfall, and humidity for that region.

We'll *train* our model by adjusting the weights slightly many times to make better predictions, using an optimization technique called *gradient descent*. Let's begin by importing Numpy and PyTorch.

In [1]:
import numpy as np
import torch

## Training data

We can represent the training data using two matrices: `inputs` and `targets`, each with one row per observation, and one column per variable.

In [2]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [3]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

We've separated the input and target variables because we'll operate on them separately. Also, we've created numpy arrays, because this is typically how you would work with training data: read some CSV files as numpy arrays, do some processing, and then convert them to PyTorch tensors.

Let's convert the arrays to PyTorch tensors.

In [4]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Linear regression model from scratch

The weights and biases (`w11, w12,... w23, b1 & b2`) can also be represented as matrices, initialized as random values. The first row of `w` and the first element of `b` are used to predict the first target variable, i.e., yield of apples, and similarly, the second for oranges.

In [5]:
# Weights and biases
w = torch.randn(2, 3, requires_grad = True)
b = torch.randn(2, requires_grad = True)
w, b

(tensor([[-0.6480, -0.2108,  0.3921],
         [-0.0154, -0.0368, -0.9665]], requires_grad=True),
 tensor([-1.0559, -0.4317], requires_grad=True))

`torch.randn` creates a tensor with the given shape, with elements picked randomly from a [normal distribution](https://jovian.ai/outlink?url=https%3A%2F%2Fen.wikipedia.org%2Fwiki%2FNormal_distribution) with mean 0 and standard deviation 1.

Our *model* is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

![](https://i.imgur.com/WGXLFvA.png)

We can define the model as follows:

In [6]:
def model(x):
    return x @ w.T + b

`@` represents matrix multiplication in PyTorch, and the `.t` or `.T` method returns the transpose of a tensor.

The matrix obtained by passing the input data into the model is a set of predictions for the target variables.

In [7]:
# Generate predictions
preds = model(inputs)
preds

tensor([[-45.6211, -45.5826],
        [-53.4771, -66.9298],
        [-62.9370, -62.7608],
        [-61.7043, -39.3481],
        [-38.5561, -72.6839]], grad_fn=<AddBackward0>)

Let's compare the predictions of our model with the actual targets.

In [8]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

You can see a big difference between our model's predictions and the actual targets because we've initialized our model with random weights and biases. Obviously, we can't expect a randomly initialized model to just work.

## Loss function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets using the following method:

- Calculate the difference between the two matrices (preds and targets).
- Square all elements of the difference matrix to remove negative values.
- Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [9]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

`torch.sum` returns the sum of all the elements in a tensor. The `.numel` method of a tensor returns the number of elements in a tensor. Let's compute the mean squared error for the current predictions of our model.

In [10]:
# Compute loss
loss = mse(preds, targets)
loss

tensor(21101.0312, grad_fn=<DivBackward0>)

Here’s how we can interpret the result: *On average, each element in the prediction differs from the actual target by the square root of the loss*. And that’s pretty bad, considering the numbers we are trying to predict are themselves in the range 50–200. The result is called the *loss* because it indicates how bad the model is at predicting the target variables. It represents information loss in the model: the lower the loss, the better the model.

## Compute gradients

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases because they have `requires_grad` set to `True`.

In [11]:
# Compute Gradients
loss.backward()

The gradients are stored in the `.grad` property of the respective tensors. Note that the derivative of the loss w.r.t. the weights matrix is itself a matrix with the same dimensions.

In [12]:
# Gradients for weights
w, w.grad

(tensor([[-0.6480, -0.2108,  0.3921],
         [-0.0154, -0.0368, -0.9665]], requires_grad=True),
 tensor([[-10757.8975, -12042.1660,  -7306.9150],
         [-12352.8037, -14087.6836,  -8662.8867]]))

## Adjust weights and biases to reduce the loss

The loss is a quadratic function of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. An important insight from calculus is that the gradient indicates the rate of change of the loss, i.e., the loss function's slope w.r.t. the weights and biases.

If a gradient element is positive:

- increasing the weight element's value slightly will increase the loss
- decreasing the weight element's value slightly will decrease the loss

![](https://i.imgur.com/WLzJ4xP.png)

If a gradient element is negative:

- increasing the weight element's value slightly will decrease the loss
- decreasing the weight element's value slightly will increase the loss

![](https://i.imgur.com/dvG2fxU.png)

The increase or decrease in the loss by changing a weight element is proportional to the gradient of the loss w.r.t. that element. This observation forms the basis of the gradient descent optimization algorithm that we'll use to improve our model (*by descending along the gradient*).

We can subtract from each weight element a small quantity proportional to the derivative of the loss w.r.t. that element to reduce the loss slightly.

In [13]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

We multiply the gradients with a very small number (`10^-5` in this case) to ensure that we don't modify the weights by a very large amount. We want to take a small step in the downhill direction of the gradient, not a giant leap. This number is called the *learning rate* of the algorithm.

We use `torch.no_grad` to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases.

In [14]:
w, b

(tensor([[-0.5404, -0.0904,  0.4652],
         [ 0.1081,  0.1041, -0.8799]], requires_grad=True),
 tensor([-1.0546, -0.4302], requires_grad=True))

In [15]:
# Let's verify that the loss is actually lower
preds = model(inputs)
loss = mse(preds, targets)
loss

tensor(14360.7754, grad_fn=<DivBackward0>)

Before we proceed, we reset the gradients to zero by invoking the `.zero_()` method. We need to do this because PyTorch accumulates gradients. Otherwise, the next time we invoke `.backward` on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results.

In [16]:
w.grad.zero_()
b.grad.zero_()
w.grad, b.grad

(tensor([[0., 0., 0.],
         [0., 0., 0.]]),
 tensor([0., 0.]))

## Train the model using gradient descent

As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can train the model using the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

Let's implement the above step by step.

In [17]:
# Generative predictions
preds = model(inputs)
preds

tensor([[-26.5564, -23.3998],
        [-28.4126, -37.7458],
        [-33.2018, -28.1104],
        [-42.8483, -17.4837],
        [-14.4565, -44.5707]], grad_fn=<AddBackward0>)

In [18]:
# Calculate the loss
loss = mse(preds, targets)
loss

tensor(14360.7754, grad_fn=<DivBackward0>)

In [19]:
# Compute gradients
loss.backward()
w.grad, b.grad

(tensor([[ -8788.7490,  -9923.7871,  -6000.2759],
         [-10060.8760, -11620.3594,  -7141.2354]]),
 tensor([-105.2951, -122.2621]))

Let's update the weights and biases using the gradients computed above.

In [20]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

    w.grad.zero_()
    b.grad.zero_()

Let's take a look at the new weights and biases.

In [21]:
w, b

(tensor([[-0.4525,  0.0088,  0.5252],
         [ 0.2087,  0.2203, -0.8085]], requires_grad=True),
 tensor([-1.0536, -0.4290], requires_grad=True))

With the new weights and biases, the model should have a lower loss.

In [22]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
loss

tensor(9817.1865, grad_fn=<DivBackward0>)

We have already achieved a significant reduction in the loss merely by adjusting the weights and biases slightly using gradient descent.

## Train for multiple epochs

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an *epoch*. Let's train the model for 100 epochs.

In [23]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

Once again, let's verify that the loss is now lower:

In [24]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
loss

tensor(179.0482, grad_fn=<DivBackward0>)

The loss is now much lower than its initial value. Let's look at the model's predictions and compare them with the targets.

In [25]:
# Predictions
preds

tensor([[ 59.0233,  75.0395],
        [ 84.9754,  94.4582],
        [109.4061, 139.4801],
        [ 33.0554,  63.9750],
        [ 99.5420,  92.5145]], grad_fn=<AddBackward0>)

In [26]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

The predictions are now quite close to the target variables. We can get even better results by training for a few more epochs.

## Linear regression using PyTorch built-ins

We've implemented linear regression & gradient descent model using some basic tensor operations. However, since this is a common pattern in deep learning, PyTorch provides several built-in functions and classes to make it easy to create and train models with just a few lines of code.

Let's begin by importing the `torch.nn` package from PyTorch, which contains utility classes for building neural networks.

In [27]:
import torch.nn as nn

As before, we represent the inputs and targets as matrices.

In [28]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [29]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

We are using 15 training examples to illustrate hoe to work with large datasets in small batches.

## Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [30]:
from torch.utils.data import TensorDataset

In [31]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation (`[0:3]` in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [32]:
from torch.utils.data import DataLoader

In [33]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

We can use the data loader in a `for` loop.

In [34]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[103.,  43.,  36.],
        [ 87., 134.,  58.],
        [ 73.,  66.,  44.],
        [ 68.,  96.,  71.],
        [ 68.,  97.,  70.]])
tensor([[ 20.,  38.],
        [119., 133.],
        [ 57.,  69.],
        [104., 118.],
        [102., 120.]])


In each iteration, the data loader returns one batch of data with the given batch size. If `shuffle` is set to `True`, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [35]:
# Define model
model = nn.Linear(3, 2)
model.weight, model.bias

(Parameter containing:
 tensor([[-0.2375,  0.1250,  0.2450],
         [-0.5572,  0.4403,  0.4337]], requires_grad=True),
 Parameter containing:
 tensor([-0.0580, -0.2719], requires_grad=True))

PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [36]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.2375,  0.1250,  0.2450],
         [-0.5572,  0.4403,  0.4337]], requires_grad=True),
 Parameter containing:
 tensor([-0.0580, -0.2719], requires_grad=True)]

We can use the model to generate predictions in the same way as before.

In [37]:
# Generate predictions
preds = model(inputs)
preds

tensor([[  1.5156,   7.2029],
        [  5.0115,  15.5278],
        [ 10.2430,  35.4083],
        [ -9.8442, -22.1257],
        [ 12.7080,  33.9111],
        [  1.1531,   6.2054],
        [  5.1316,  15.5212],
        [ 10.2506,  35.2849],
        [ -9.4816, -21.1282],
        [ 13.1906,  34.9020],
        [  1.6356,   7.1963],
        [  4.6490,  14.5303],
        [ 10.1230,  35.4149],
        [-10.3268, -23.1166],
        [ 13.0706,  34.9086]], grad_fn=<AddmmBackward>)

## Loss function

Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [38]:
# Import nn.functional
import torch.nn.functional as F

The `nn.functional` package contains many useful loss functions and several other utilities.

In [39]:
# Define loss function
loss_fn = F.mse_loss

Let's compute the loss for the current predictions of our model.

In [40]:
loss = loss_fn(model(inputs), targets)
loss

tensor(6108.8911, grad_fn=<MseLossBackward>)

## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD is short for "stochastic gradient descent". The term *stochastic* indicates that samples are selected in random batches instead of as a single group.

In [41]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Note that `model.parameters()` is passed as an argument to `optim.SGD` so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

## Train the model

We are now ready to train the model. We'll follow the same process to implement gradient descent:

- Generate predictions

- Calculate the loss

- Compute gradients w.r.t the weights and biases

- Adjust the weights by subtracting a small quantity proportional to the gradient

- Reset the gradients to zero

The only change is that we'll work on batches of data instead of processing the entire training data in every iteration. Let's define a utility function `fit` that trains the model for a given number of epochs.

In [42]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb, yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
            
        # Print the progress
        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Some things to note above:

- We use the data loader defined earlier to get batches of data for every iteration.

- Instead of updating parameters (weights and biases) manually, we use `opt.step` to perform the update and `opt.zero_grad` to reset the gradients to zero.

- We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. `loss.item` returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [43]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 152.5999
Epoch [20/100], Loss: 31.2272
Epoch [30/100], Loss: 71.0316
Epoch [40/100], Loss: 32.5556
Epoch [50/100], Loss: 20.8341
Epoch [60/100], Loss: 25.4347
Epoch [70/100], Loss: 19.3376
Epoch [80/100], Loss: 15.7897
Epoch [90/100], Loss: 8.5012
Epoch [100/100], Loss: 6.0743


Let's generate predictions using our model and verify that they're close to our targets.

In [44]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 57.7544,  70.5707],
        [ 81.6282,  99.4921],
        [117.7857, 135.0318],
        [ 25.4765,  38.8062],
        [ 98.4300, 116.0038],
        [ 56.6149,  69.4816],
        [ 81.3611,  99.3809],
        [118.0455, 135.5269],
        [ 26.6161,  39.8953],
        [ 99.3024, 116.9817],
        [ 57.4874,  70.4595],
        [ 80.4886,  98.4030],
        [118.0528, 135.1430],
        [ 24.6041,  37.8283],
        [ 99.5695, 117.0929]], grad_fn=<AddmmBackward>)

In [45]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

Indeed, the predictions are quite close to our targets. We have a trained a reasonably good model to predict crop yields for apples and oranges by looking at the average temperature, rainfall, and humidity in a region. We can use it to make predictions of crop yields for new regions by passing a batch containing a single row of input.

In [46]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[54.3750, 67.4336]], grad_fn=<AddmmBackward>)

The predicted yield of apples is 54.8 tons per hectare, and that of oranges is 68.5 tons per hectare.

In [47]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.3063,  0.8332,  0.5662],
         [-0.2414,  0.8477,  0.7365]], requires_grad=True),
 Parameter containing:
 tensor([-0.0566, -0.2676], requires_grad=True)]

## Further Reading

Here are some resources for learning more about linear regression and gradient descent:

- An visual & animated explanation of gradient descent: https://www.youtube.com/watch?v=IHZwWFHWa-w

- For a more detailed explanation of derivates and gradient descent, see [these notes from a Udacity course](https://jovian.ai/outlink?url=https%3A%2F%2Fstorage.googleapis.com%2Fsupplemental_media%2Fudacityu%2F315142919%2FGradient%2520Descent.pdf).

- For an animated visualization of how linear regression works, [see this post](https://jovian.ai/outlink?url=https%3A%2F%2Fhackernoon.com%2Fvisualizing-linear-regression-with-pytorch-9261f49edb09).

- For a more mathematical treatment of matrix calculus, linear regression and gradient descent, you should check out [Andrew Ng's excellent course notes](https://jovian.ai/outlink?url=https%3A%2F%2Fgithub.com%2FCleo-Stanford-CS%2FCS229_Notes%2Fblob%2Fmaster%2Flectures%2Fcs229-notes1.pdf) from CS229 at Stanford University.

To practice and test your skills, you can participate in the [Boston Housing Price Prediction](https://jovian.ai/outlink?url=https%3A%2F%2Fwww.kaggle.com%2Fc%2Fboston-housing) competition on Kaggle.

## Exercise- 

### Q1. What is a linear regression model? Give an example of a problem formulated as a linear regression model.

**Ans:** Linear regression is a basic and commonly used type of predictive analysis.  The overall idea of regression is to examine two things: 
1. does a set of predictor variables do a good job in predicting an outcome (dependent) variable?  
2. Which variables in particular are significant predictors of the outcome variable, and in what way do they–indicated by the magnitude and sign of the beta estimates–impact the outcome variable?  

These regression estimates are used to explain the relationship between one dependent variable and one or more independent variables.  

The simplest form of the regression equation with one dependent and one independent variable is defined by the formula:
$\ y = c + b*x\$

where y = estimated dependent variable score, c = constant, b = regression coefficient, and x = score on the independent variable.

ex- Hosing price prediction.

### Q2. What are input and target variables in a dataset? Give an example.

The Boston Housing Dataset

The Boston Housing Dataset is a derived from information collected by the U.S. Census Service concerning housing in the area of Boston MA. The following describes the dataset columns:

- CRIM - per capita crime rate by town
- ZN - proportion of residential land zoned for lots over 25,000 sq.ft.
- INDUS - proportion of non-retail business acres per town.
- CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)
- NOX - nitric oxides concentration (parts per 10 million)
- RM - average number of rooms per dwelling
- AGE - proportion of owner-occupied units built prior to 1940
- DIS - weighted distances to five Boston employment centres
- RAD - index of accessibility to radial highways
- TAX - full-value property-tax rate per $\$10,000$
- PTRATIO - pupil-teacher ratio by town
- B - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
- LSTAT - % lower status of the population
- MEDV - Median value of owner-occupied homes in $1000's

Ans: The dependent variable is dataset is called target while independent variable is called input or predictor variable

Ex:- In above dataset description **CRIM, ZN, INDUS, CHAS, NOX, RM, AGE, DIS, RAD, TAX, PTRATIO, B, LSTAT** are the input variables and **MEDV** is target variable.

In [48]:
import pandas as pd 

In [122]:
data = pd.read_csv('housing.csv', delim_whitespace=' ', header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [123]:
input = data.iloc[:, :-1]
target = data.iloc[:, -1:]

In [124]:
input = torch.tensor(input.values.astype(np.float32))
input

tensor([[6.3200e-03, 1.8000e+01, 2.3100e+00,  ..., 1.5300e+01, 3.9690e+02,
         4.9800e+00],
        [2.7310e-02, 0.0000e+00, 7.0700e+00,  ..., 1.7800e+01, 3.9690e+02,
         9.1400e+00],
        [2.7290e-02, 0.0000e+00, 7.0700e+00,  ..., 1.7800e+01, 3.9283e+02,
         4.0300e+00],
        ...,
        [6.0760e-02, 0.0000e+00, 1.1930e+01,  ..., 2.1000e+01, 3.9690e+02,
         5.6400e+00],
        [1.0959e-01, 0.0000e+00, 1.1930e+01,  ..., 2.1000e+01, 3.9345e+02,
         6.4800e+00],
        [4.7410e-02, 0.0000e+00, 1.1930e+01,  ..., 2.1000e+01, 3.9690e+02,
         7.8800e+00]])

In [125]:
target = torch.tensor(target.values.astype(np.float32))
target

tensor([[24.0000],
        [21.6000],
        [34.7000],
        [33.4000],
        [36.2000],
        [28.7000],
        [22.9000],
        [27.1000],
        [16.5000],
        [18.9000],
        [15.0000],
        [18.9000],
        [21.7000],
        [20.4000],
        [18.2000],
        [19.9000],
        [23.1000],
        [17.5000],
        [20.2000],
        [18.2000],
        [13.6000],
        [19.6000],
        [15.2000],
        [14.5000],
        [15.6000],
        [13.9000],
        [16.6000],
        [14.8000],
        [18.4000],
        [21.0000],
        [12.7000],
        [14.5000],
        [13.2000],
        [13.1000],
        [13.5000],
        [18.9000],
        [20.0000],
        [21.0000],
        [24.7000],
        [30.8000],
        [34.9000],
        [26.6000],
        [25.3000],
        [24.7000],
        [21.2000],
        [19.3000],
        [20.0000],
        [16.6000],
        [14.4000],
        [19.4000],
        [19.7000],
        [20.5000],
        [25.

### Q3. What are weights and biases in a linear regression model?

Ans:

**Weight** - Weight is the strength of the connection. If I increase the input then how much influence does it have on the output.

**Bias** - as means how far off our predictions are from real values.

### Q4. How do you represent tabular data using PyTorch tensors?

In [144]:
# import pandas as pd 

# data = pd.read_csv('housing.csv', delim_whitespace=' ', header=None)
# data.head()

# input = data.iloc[:, :-1]
# target = data.iloc[:, -1:]

# input = torch.tensor(input.values.astype(np.float32))
# input

# target = torch.tensor(target.values.astype(np.float32))
# target

### Q5. Why do we create separate matrices for inputs and targets while training a linear regression model?

Ans: To build our model correctly without disturbing output or target variable and easy calculations like during accuracy comparison.

### Q6. How do you determine the shape of the weights matrix & bias vector given some training data?

Ans: It depends on the no. of features, output and type of network you want to build.

ex- In boston housing price predictions we have 13 features and 1 output variable so for simple linear regression our weight matrix shape will be (1, 13) and bias will be of (1, 1).

### Q7. How do you create randomly initialized weights & biases with a given shape?

In [145]:
w = torch.randn(1, 13, requires_grad = True)
b = torch.randn(1, requires_grad = True)
w, b

(tensor([[ 0.2480, -0.2941,  0.1674,  0.5883, -0.7377,  0.1029,  0.8168, -0.4203,
          -1.1544,  1.0556, -0.0270,  0.7987, -0.7389]], requires_grad=True),
 tensor([-0.0887], requires_grad=True))

### Q8. How is a linear regression model implemented using matrix operations? Explain with an example.

In [146]:
def model(x):
    return x @ w.T + b

### Q9. How do you generate predictions using a linear regression model?

In [147]:
preds = model(input)
preds

tensor([[ 671.0303],
        [ 626.6833],
        [ 612.7487],
        [ 578.9152],
        [ 585.8475],
        [ 587.3077],
        [ 678.7534],
        [ 698.7872],
        [ 685.7104],
        [ 683.5519],
        [ 692.7327],
        [ 692.2111],
        [ 649.7488],
        [ 679.9258],
        [ 683.6273],
        [ 674.4949],
        [ 646.7874],
        [ 683.5679],
        [ 570.9995],
        [ 679.6152],
        [ 684.4064],
        [ 695.0481],
        [ 697.0944],
        [ 700.9874],
        [ 698.4705],
        [ 618.8132],
        [ 682.3815],
        [ 623.2155],
        [ 696.2394],
        [ 685.0545],
        [ 666.6755],
        [ 691.9011],
        [ 551.3208],
        [ 669.6241],
        [ 581.6508],
        [ 653.5284],
        [ 631.2413],
        [ 632.1476],
        [ 619.2139],
        [ 569.0978],
        [ 565.9640],
        [ 547.6849],
        [ 548.3026],
        [ 555.8825],
        [ 577.6119],
        [ 578.2914],
        [ 574.9866],
        [ 610

### Q10. Why are the predictions of a randomly initialized model different from the actual targets?

Ans: Because weights and bias are initialized randomly so predictions are randomly generated.

### Q11. What is a loss function? What does the term “loss” signify?

Ans: Loss function outputs the value which define how wrong our predictions are.

loss- Difference between the actual target and square root of the loss i.e. how our predictions are far from target.

### Q12. What is mean squared error?

Ans: Square of the difference between the actual target and the predictions divided by no of elements.

### Q13. Write a function to calculate mean squared using model predictions and actual targets.

In [148]:
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [149]:
loss = mse(preds, target)
loss

tensor(556108.0625, grad_fn=<DivBackward0>)

### Q14. What happens when you invoke the `.backward` function on the result of the mean squared error loss function?

Ans: It automatically calculates the gradient of loss w.r.t. weights and biases.

In [150]:
loss.backward()

### Q15. Why is the derivative of the loss w.r.t. the weights matrix itself a matrix? - What do its elements represent?

Ans: Suppose a PyTorch gradient enabled tensors X as:

X = **[x1, x2, ….. xn]** (Let this be the weights of some machine learning model)
X undergoes some operations to form a vector **Y**.

Y = f(X) = **[y1, y2, …. ym]**

**Y** is then used to calculate a scalar loss ***l***. Suppose a vector ***v*** happens to be the gradient of the scalar loss ***l*** with respect the vector **Y** as follows:

![](https://miro.medium.com/max/440/1*PLhFxIHfMUervwt-z8G4Iw.png)

The vector ***v*** is called the `grad_tensor` and passed to the `backward()` function as an argument.

To get the gradient of the loss ***l*** with respect to the weights **X** the Jacobian matrix **J** is vector-multiplied with the vector ***v***

![](https://miro.medium.com/max/875/1*bAOzjl1KncUmXURr2nFvBA.png)

### Q16. How is the derivate of the loss w.r.t. a weight element useful for reducing the loss? Explain with an example.

Ans: The loss is a quadratic function of our weights and biases. The increase or decrease in the loss by changing a weight element is proportional to the gradient of the loss w.r.t. that element.

### Q17. Suppose the derivative of the loss w.r.t. a weight element is positive. Should you increase or decrease the element’s value slightly to get a lower loss?

Ans: Decrease

### Q18. Suppose the derivative of the loss w.r.t. a weight element is negative. Should you increase or decrease the element’s value slightly to get a lower loss?

Ans: Increase

### Q19. How do you update the weights and biases of a model using their respective gradients to reduce the loss slightly?

In [151]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    
w, b

(tensor([[ 0.1812, -0.4301, -0.0106,  0.5873, -0.7460,  0.0123, -0.2328, -0.4715,
          -1.3167, -5.3943, -0.2984, -4.3627, -0.9349]], requires_grad=True),
 tensor([-0.1032], requires_grad=True))

### Q20. What is the gradient descent optimization algorithm? Why is it called “gradient descent”?

Ans: To make better predictions or reduce the loss we use gradient descent algorithm by increasing or decreasing the slope or gradient by means of learning rate and further updates the parameters.

It is called gradient descent because descending through gradient to update parameters or reducing the loss.

### Q21. Why do you subtract a “small quantity” proportional to the gradient from the weights & biases, not the actual gradient itself?

Ans: To ensure that we don't modify the weights by a very large amount. We want to take a small step in the downhill direction of the gradient, not a giant leap.

### Q22. What is learning rate? Why is it important?

Ans: We multiply the gradients with a very small number to ensure that we don't modify the weights by a very large amount. We want to take a small step in the downhill direction of the gradient, not a giant leap. This number is called the learning rate of the algorithm.

Because it controls how much we are adjusting the weights of our network with respect the loss gradient.

### Q23. What is `torch.no_grad`?

Ans: torch. no_grad() basically skips the gradient calculation over the weights. That means you are not changing any weight in the specified layers.

### Q24. Why do you reset gradients to zero after updating weights and biases?

Ans: Because PyTorch accumulates gradients, otherwise, the next time we invoke .backward on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results.

### Q25. What are the steps involved in training a linear regression model using gradient descent?

Ans: The following steps are involved:

- Generate predictions

- Calculate the loss

- Compute gradients w.r.t the weights and biases

- Adjust the weights by subtracting a small quantity proportional to the gradient

- Reset the gradients to zero

### Q26. What is an epoch?

Ans: No. of iterations to which our model iterates to updates the weights and biases.

### Q27. What is the benefit of training a model for multiple epochs?

Ans: It very much reduce the loss to make better predictions.

### Q28. How do you make predictions using a trained model?

In [152]:
preds = model(input)
preds

tensor([[-3364.1228],
        [-3074.6418],
        [-3047.9543],
        [-2945.3831],
        [-2959.4680],
        [-2948.2913],
        [-3450.2627],
        [-3469.2566],
        [-3435.4939],
        [-3420.8088],
        [-3451.1138],
        [-3460.8232],
        [-3424.6021],
        [-3423.8928],
        [-3357.2854],
        [-3417.1743],
        [-3370.7346],
        [-3389.9963],
        [-2949.9937],
        [-3402.1064],
        [-3355.0422],
        [-3416.0461],
        [-3440.1770],
        [-3433.0020],
        [-3427.5505],
        [-3029.1875],
        [-3349.2917],
        [-3043.5159],
        [-3396.4854],
        [-3360.2832],
        [-3284.2642],
        [-3348.8750],
        [-2729.5044],
        [-3274.1785],
        [-2794.4712],
        [-3275.8647],
        [-3191.5271],
        [-3269.0664],
        [-3252.5085],
        [-3139.0923],
        [-3135.4575],
        [-2955.9128],
        [-2948.7859],
        [-2998.6655],
        [-2986.3271],
        [-

### Q29. What should you do if your model’s loss doesn’t decrease while training? Hint: learning rate.

Ans: We try different learning rate to optimize the model.

### Q30. What is `torch.nn`?

Ans: PyTorch provides the modules and classes to help you create and train neural networks.

### Q31. What is the purpose of the `TensorDataset` class in PyTorch? Give an example.

Ans: The TensorDataset allows us to access a small section of the training data using the array indexing notation. It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.

In [153]:
train_ds = TensorDataset(input, target)
train_ds[0:5]

(tensor([[6.3200e-03, 1.8000e+01, 2.3100e+00, 0.0000e+00, 5.3800e-01, 6.5750e+00,
          6.5200e+01, 4.0900e+00, 1.0000e+00, 2.9600e+02, 1.5300e+01, 3.9690e+02,
          4.9800e+00],
         [2.7310e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01, 6.4210e+00,
          7.8900e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02, 1.7800e+01, 3.9690e+02,
          9.1400e+00],
         [2.7290e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01, 7.1850e+00,
          6.1100e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02, 1.7800e+01, 3.9283e+02,
          4.0300e+00],
         [3.2370e-02, 0.0000e+00, 2.1800e+00, 0.0000e+00, 4.5800e-01, 6.9980e+00,
          4.5800e+01, 6.0622e+00, 3.0000e+00, 2.2200e+02, 1.8700e+01, 3.9463e+02,
          2.9400e+00],
         [6.9050e-02, 0.0000e+00, 2.1800e+00, 0.0000e+00, 4.5800e-01, 7.1470e+00,
          5.4200e+01, 6.0622e+00, 3.0000e+00, 2.2200e+02, 1.8700e+01, 3.9690e+02,
          5.3300e+00]]),
 tensor([[24.0000],
         [21.6000],
         [34.7000],
   

### Q32. What is a data loader in PyTorch? Give an example.

Ans: A DataLoader is a module, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [154]:
from torch.utils.data import DataLoader

batch_size = 10
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

### Q33. How do you use a data loader to retrieve batches of data?

In [155]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[3.2982e-01, 0.0000e+00, 2.1890e+01, 0.0000e+00, 6.2400e-01, 5.8220e+00,
         9.5400e+01, 2.4699e+00, 4.0000e+00, 4.3700e+02, 2.1200e+01, 3.8869e+02,
         1.5030e+01],
        [5.4452e-01, 0.0000e+00, 2.1890e+01, 0.0000e+00, 6.2400e-01, 6.1510e+00,
         9.7900e+01, 1.6687e+00, 4.0000e+00, 4.3700e+02, 2.1200e+01, 3.9690e+02,
         1.8460e+01],
        [6.8880e-02, 0.0000e+00, 2.4600e+00, 0.0000e+00, 4.8800e-01, 6.1440e+00,
         6.2200e+01, 2.5979e+00, 3.0000e+00, 1.9300e+02, 1.7800e+01, 3.9690e+02,
         9.4500e+00],
        [9.0600e-03, 9.0000e+01, 2.9700e+00, 0.0000e+00, 4.0000e-01, 7.0880e+00,
         2.0800e+01, 7.3073e+00, 1.0000e+00, 2.8500e+02, 1.5300e+01, 3.9472e+02,
         7.8500e+00],
        [5.2018e+00, 0.0000e+00, 1.8100e+01, 1.0000e+00, 7.7000e-01, 6.1270e+00,
         8.3400e+01, 2.7227e+00, 2.4000e+01, 6.6600e+02, 2.0200e+01, 3.9543e+02,
         1.1480e+01],
        [4.4223e+00, 0.0000e+00, 1.8100e+01, 0.0000e+00, 5.8400e-01, 6.0030e+00,

### Q34. What are the benefits of shuffling the training data before creating batches?

Ans: It shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

### Q35. What is the benefit of training in small batches instead of training with the entire dataset?

Ans: Due to computational power restrictions we use batches of dataset to reduce the training time as well as memory use.

### Q36. What is the purpose of the `nn.Linear` class in PyTorch? Give an example.

In [157]:
model = nn.Linear(13, 1)

### Q37. How do you see the weights and biases of a `nn.Linear` model?

In [158]:
model.weight, model.bias

(Parameter containing:
 tensor([[-0.0760, -0.0086, -0.1135,  0.2390, -0.1273,  0.1106,  0.1965, -0.2126,
           0.1807,  0.2618, -0.0662,  0.0278,  0.2144]], requires_grad=True),
 Parameter containing:
 tensor([0.1167], requires_grad=True))

### Q38. What is the purpose of the `torch.nn.functional` module?

Ans: The nn.functional package contains many useful loss functions and several other utilities.

### Q39. How do you compute mean squared error loss using a PyTorch built-in function?

In [159]:
import torch.nn.functional as F

loss_fn = F.mse_loss
loss = loss_fn(model(input), target)
loss

tensor(14708.8125, grad_fn=<MseLossBackward>)

### Q40. What is an optimizer in PyTorch?

Ans: Instead of manually manipulating the model's weights & biases using gradients, we can use the pytorch's built-in optimizes. 

### Q41. What is `torch.optim.SGD`? What does SGD stand for?

Ans: torch.optim.SGD is a pytorch's built-in optimizer. 

SGD - Stochastic Gradient Descent. The term stochastic indicates that samples are selected in random batches instead of as a single group.

### Q42. What are the inputs to a PyTorch optimizer?

Ans: models parameters, momentum and learning rate.

### Q43. Give an example of creating an optimizer for training a linear regression model.

In [198]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

### Q44. Write a function to train a `nn.Linear` model in batches using gradient descent.

In [199]:
def fit(num_epochs, model, loss_fn, optimizer, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb, yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
            
        # Print the progress
        if (epoch+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [200]:
fit(400, model, loss_fn, optimizer, train_dl)

Epoch [100/400], Loss: 16037.1631
Epoch [200/400], Loss: 19136.3633
Epoch [300/400], Loss: 20512.0859
Epoch [400/400], Loss: 7498.4043


### Q45. How do you use a linear regression model to make predictions on previously unseen data?

In [201]:
model(torch.tensor([[0.00561, 20.0, 4.31, 0, 0.414, 5.515, 67.2, 4.4500, 1, 301.0, 14.3, 401.90, 3.82]]))

tensor([[102.2728]], grad_fn=<AddmmBackward>)